Installing libraries:

In [2]:

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-texeurj2/unsloth_d4e6870dff4146ffa787cc77f7225268
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-texeurj2/unsloth_d4e6870dff4146ffa787cc77f7225268
  Resolved https://github.com/unslothai/unsloth.git to commit b9067f98d124173b354808b116f931410e5b699c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Import library:

In [3]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
import pandas as pd
import json, yaml
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


# 🚀 Fine-Tuning LLMs with Unsloth and 4-Bit Quantization on Google Colab

This notebook sets up all required libraries and functions to fine-tune a large language model (LLM) using the [Unsloth](https://github.com/unslothai/unsloth) library.  
We will select a model from the Unsloth repository and load it into memory in 4-bit quantized format to significantly improve training performance and memory efficiency on Colab's T4 GPU.

In [4]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "**************************", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.4.8: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

## ⚙️ Setting Up PEFT with LoRA for Efficient Fine-Tuning

In this step, we configure **PEFT (Parameter-Efficient Fine-Tuning)** using **LoRA (Low-Rank Adaptation)** to fine-tune the `LLaMA 3.1–8B` model with significantly reduced memory usage and training cost.

We use the `FastLanguageModel.get_peft_model()` method from **Unsloth** to wrap our base model with LoRA.

### 🔍 Key Configurations:
- `r = 16`: The rank of the LoRA matrices. Common choices are 8, 16, 32, etc. Higher values increase capacity but use more memory.
- `target_modules`: Specifies which layers to apply LoRA to. Here, we target key transformer components like `q_proj`, `k_proj`, etc.
- `lora_alpha = 16`: Scaling factor to adjust the impact of LoRA layers.
- `lora_dropout = 0`: Dropout rate for LoRA; 0 is optimal for speed.
- `bias = "none"`: We avoid training biases to reduce overhead.
- `use_gradient_checkpointing = "unsloth"`: Saves ~30% VRAM and allows longer context windows with minimal slowdown.
- `random_state = 3407`: Ensures reproducibility.
- `use_rslora` and `loftq_config`: Optional features (e.g., Rank-Stabilized LoRA, LoftQ) left disabled for simplicity.

This setup allows us to fine-tune powerful models like LLaMA 3.1–8B even on resource-limited hardware like Colab’s T4 GPU.

Now let’s apply this configuration to our model:

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `base_model.model.vision_tower.vision_model` require gradients


## 🧹 Data Preparation for Fine-Tuning

Now that we’ve completed the model setup and LoRA fine-tuning configuration, it's time to prepare the dataset that we will train the model on.

📄 **Dataset Format**:  
We already have a dataset in **CSV format**, where each row contains a dictionary-like structure with:
- `instruction`: The task or goal given to the model
- `input`: The user's input or context
- `output`: The expected response from the model

### 🧠 Instruction-Tuning Format:
Each data point will be wrapped using the **Alpaca-style instruction prompt template**, which helps the model better understand the task structure during fine-tuning.

<span>
The formatted prompt typically looks like this:

  Instruction:

    {instruction}

Input:

    {input}

Response:

    {output}<|eos|>
</span>

We also append an **EOS (End-of-Sequence)** token at the end of each sample to help the model understand where each training example ends.

This structured formatting enables more robust and generalizable fine-tuning behavior, especially for instruction-following models like LLaMA.

Let’s now convert the raw data into this template and feed it into the model.

In [6]:
import json
import pandas as pd

# Read JSONL file line by line
with open('../dataset/alpaca_finetune_data.jsonl', 'r') as f:
    lines = f.readlines()
    json_f = [json.loads(line) for line in lines]

# Convert to DataFrame
df = pd.DataFrame(json_f)
print(df.columns)

Index(['instruction', 'input', 'output'], dtype='object')


In [7]:
from datasets import Dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched=True)
# (Optional) View one formatted sample
print(dataset[0]['text'])

Map:   0%|          | 0/2025 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Extract product attributes from the input query.

### Input:
شال کشی

### Response:
{"Category": ["scarf"], "Persian-Product-Name": ["شال"], "Gender": [], "English-Query": ["Stretchy shawl"], "Color": [], "Similar-Keywords": ["scarf", "stretchy"]}<end_of_turn>


In [16]:
dataset[23]["text"]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExtract product attributes from the input query.\n\n### Input:\nشلوار پارچه مردانه\n\n### Response:\n{"Category": ["pants"], "Persian-Product-Name": ["شلوار"], "Gender": ["Men"], "English-Query": ["Men\'s fabric pants"], "Color": [], "Similar-Keywords": ["men\'s trousers", "fabric pants"]}<|end_of_text|>'

In [7]:
len(dataset)

2025

In [17]:
EOS_TOKEN

'<|end_of_text|>'

We wrap these into a single text field and pass them to the trainer.

---

### ⚙️ Trainer Configuration

We configure training with the following:

| Parameter                    | Description                                                                 |
|-----------------------------|-----------------------------------------------------------------------------|
| `per_device_train_batch_size` | Small batch size to fit into memory                                         |
| `gradient_accumulation_steps` | Accumulate gradients to simulate a larger batch size                        |
| `learning_rate`             | A small learning rate with a linear decay scheduler                         |
| `max_steps`                 | Number of steps to train (can be replaced with `num_train_epochs`)          |
| `fp16` / `bf16`             | Use mixed-precision to accelerate training (depending on hardware support)  |
| `optim`                     | Memory-efficient optimizer (`adamw_8bit`)                                   |
| `seed`                      | Ensures reproducibility                                                     |
| `output_dir`                | Directory where model checkpoints and logs will be saved                    |

---

### 🧠 Training the Model

```

In [8]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        max_steps = 500,
        learning_rate = 1e-3,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)
trainer_stats = trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2025 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,025 | Num Epochs = 2 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,788,480/4,000,000,000 (0.82% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mhm_sadegh to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,3.954900
2,3.838100
3,3.183400
4,2.326000
5,1.722400
6,1.018700
7,0.790100
8,0.596900
9,0.564500
10,0.586300


 Inference with Fine-Tuned Model

Once our model has been fine-tuned (which typically takes 5–10 minutes on a single GPU with ~12GB of RAM), we are ready to generate predictions (inference) using any custom input.

Below is the code used for performing inference using the fine-tuned model:

In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Extract product attributes from the input query.", # instruction
        "تیشرت میکی موس پسرونه", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True )
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExtract product attributes from the input query.\n\n### Input:\nتیشرت میکی موس پسرونه\n\n### Response:\n{"Category": ["blouse"], "Persian-Product-Name": ["تیشرت"], "Gender": ["Men"], "English-Query": ["boys\' Mickey Mouse T-shirt"], "Color": [], "Similar-Keywords": ["T-shirt", "Boys"]}<end_of_turn>']

✅ Final Step: Saving or Sharing the Fine-Tuned Model

Awesome! 🎉
We’ve successfully fine-tuned the LLaMA 3.1–8B model using our custom dataset (2000 examples) — all done using Unsloth + Google Colab, without any paid resources.
We also verified the fine-tuning by generating a test prediction using the newly trained model.

💾 Option 1: Save Locally

You can save your fine-tuned model locally in the Colab session:

In [ ]:
model.save_pretrained_merged(
    "amrs_csv_gen_model",              # Folder to save the model
    tokenizer,                         # Tokenizer to save
    save_method = "merged_16bit",      # Saves in a compact format
)

☁️ Option 2: Push to Hugging Face Hub

If you’d like to publish and reuse your model easily (or share it with the community), you can push it to your Hugging Face account:

In [3]:
model.push_to_hub_merged(
    "mhmsadegh/gemma-3-4b-it-fashion-query-rewriting",        # Replace with your HF username/model name
    tokenizer,
    save_method = "merged_16bit",
    token = "**************************" # Your Hugging Face access token
)